In [1]:
!nvidia-smi

Wed Jul 13 12:14:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      On   | 00000000:01:00.0 Off |                   On |
| N/A   33C    P0    45W / 275W |                  N/A |     N/A      Default |
|                               |                      |              Enabled |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| MIG d

In [2]:
# 활용 gpu idx 세팅
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# import package

In [7]:
import json
from pathlib import Path

import numpy as np
import pandas as pd
from scipy import sparse
from tqdm import tqdm

# model import
import torch
# from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('./input/AI4Code')
# data_dir = Path(f'./data/ai4code/')
print(data_dir)

input/AI4Code


# 필요 함수 정의

In [4]:
# Additional code cells

#preprocess.py -11
def clean_code(cell):
    return str(cell).replace("\\n", "\n")


def sample_cells(cells, n):
    cells = [clean_code(cell) for cell in cells]
    if n >= len(cells):
        return [cell[:200] for cell in cells]
    else: #code cell 개수가 지정된 n 보다 넘어가면
        results = []
        step = len(cells) / n #ex) 25/20 = 1.25 씩 뛰어 넘으면서 셀을 추가
        idx = 0
        while int(np.round(idx)) < len(cells):
            results.append(cells[int(np.round(idx))])
            idx += step
        assert cells[0] in results # 마지막 셀 꼭 추가
        if cells[-1] not in results:
            results[-1] = cells[-1]
        return results


def get_features(df):
    features = dict()
    df = df.sort_values("rank").reset_index(drop=True)
    for idx, sub_df in tqdm(df.groupby("id")): # 각 노트북에 대한 정보 저장
        features[idx] = dict()
        total_md = sub_df[sub_df.cell_type == "markdown"].shape[0]
        code_sub_df = sub_df[sub_df.cell_type == "code"]
        total_code = code_sub_df.shape[0]
        codes = sample_cells(code_sub_df.source.values, 20)
        features[idx]["total_code"] = total_code
        features[idx]["total_md"] = total_md
        features[idx]["codes"] = codes
    return features

In [5]:
#metric.py
from bisect import bisect

def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max

def read_notebook(path):
    return (
        pd.read_json(
            path,
            dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)
            .rename_axis('cell_id')
    )

In [6]:
#model.py
from tqdm import tqdm
import sys, os
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
import torch.nn as nn
import torch

class MarkdownModel(nn.Module):
    def __init__(self, model_path):
        super(MarkdownModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_path)
        self.top = nn.Linear(769, 1)
        
    def forward(self, ids, mask, fts):
        x = self.model(ids, mask)[0]
        x = self.top(torch.cat((x[:, 0, :], fts),1))
        return x


#dataset.py
from torch.utils.data import DataLoader, Dataset

class MarkdownDataset(Dataset):

    def __init__(self, df, model_name_or_path, total_max_len, md_max_len, fts):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.md_max_len = md_max_len
        self.total_max_len = total_max_len  # maxlen allowed by model config
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        self.fts = fts

    def __getitem__(self, index):
        row = self.df.iloc[index]

        inputs = self.tokenizer.encode_plus(
            row.source,
            None,
            add_special_tokens=True,
            max_length=self.md_max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        code_inputs = self.tokenizer.batch_encode_plus(
            [str(x) for x in self.fts[row.id]["codes"]],
            add_special_tokens=True,
            max_length=24,
            padding="max_length",
            truncation=True
        )
        n_md = self.fts[row.id]["total_md"]
        n_code = self.fts[row.id]["total_md"]
        if n_md + n_code == 0:
            fts = torch.FloatTensor([0])
        else:
            fts = torch.FloatTensor([n_md / (n_md + n_code)])

        ids = inputs['input_ids']
        for x in code_inputs['input_ids']:
            ids.extend(x[:-1])
        ids = ids[:self.total_max_len]
        if len(ids) != self.total_max_len:
            ids = ids + [self.tokenizer.pad_token_id, ] * (self.total_max_len - len(ids))
        ids = torch.LongTensor(ids)

        mask = inputs['attention_mask']
        for x in code_inputs['attention_mask']:
            mask.extend(x[:-1])
        mask = mask[:self.total_max_len]
        if len(mask) != self.total_max_len:
            mask = mask + [self.tokenizer.pad_token_id, ] * (self.total_max_len - len(mask))
        mask = torch.LongTensor(mask)

        assert len(ids) == self.total_max_len

        return ids, mask, fts, torch.FloatTensor([row.pct_rank])

    def __len__(self):
        return self.df.shape[0]

# Valid 시작!

In [8]:
# Train 데이터(notebook)의 order 순서가 적혀있는 데이터(train_orders.csv) import
# 여기서의 id는 notebook 단위

#preprocess.py -2
df_orders = pd.read_csv(
    data_dir / 'train_orders.csv',
    index_col='id',
    squeeze=True,
).str.split()  # cell_id가 텍스트로 붙어있음, 띄어쓰기 단위로 끊어서 리스트화

print(df_orders.shape)
df_orders.head(2)

/tmp/ipykernel_96934/979864248.py:5: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  df_orders = pd.read_csv(


(139256,)


id
00001756c60be8    [1862f0a6, 448eb224, 2a9e43d6, 7e2f170a, 038b763d, 77e56113, 2eefe0ef, 1ae087ab, 0beab1cd, 8ffe0b25, 9a78ab76, 0d136...
00015c83e2717b    [2e94bd7a, 3e99dee9, b5e286ea, da4f7550, c417225b, 51e3cd89, 2600b4eb, 75b65993, cf195f8b, 25699d02, 72b3201a, f2c75...
Name: cell_order, dtype: object

In [9]:
# 학습데이터 경로
experiment_data_dir = f'./ai4code-999/'

In [10]:
train_df_mark = pd.read_csv(f'{experiment_data_dir}/train_mark.csv')#.drop("parent_id", axis=1).dropna().reset_index(drop=True)
train_fts = json.load(open(f'{experiment_data_dir}/train_fts.json'))
train_df = pd.read_csv(f'{experiment_data_dir}/train.csv')
val_df_mark = pd.read_csv(f'{experiment_data_dir}/val_mark.csv')#.drop("parent_id", axis=1).dropna().reset_index(drop=True)
val_fts = json.load(open(f'{experiment_data_dir}/val_fts.json'))
val_df = pd.read_csv(f'{experiment_data_dir}/val.csv')

In [11]:
#이후  0713
print(train_df.shape, val_df.shape)

(6364816, 8) (5834, 8)


In [12]:
#이후  0713
print(train_df_mark.shape, val_df_mark.shape)

(2164091, 8) (1977, 8)


In [30]:
# tokenizer dir 지정
tokenizer_path = f'./model/finetuned/20220703/outputs'

In [24]:
md_max_len = 64
total_max_len = 512
batch_size = 8
# accumulation_steps = 4
# epochs = 4
n_workers = 8

train_ds = MarkdownDataset(train_df_mark, model_name_or_path=tokenizer_path, md_max_len=md_max_len,
                           total_max_len=total_max_len, fts=train_fts)
val_ds = MarkdownDataset(val_df_mark, model_name_or_path=tokenizer_path, md_max_len=md_max_len,
                         total_max_len=total_max_len, fts=val_fts)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=n_workers,
                          pin_memory=False, drop_last=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=n_workers,
                        pin_memory=False, drop_last=False)

In [6]:
def read_data(data):
    return tuple(d.to(device) for d in data[:-1]), data[-1].to(device)

def validate(model, val_loader):
    model.eval()

    tbar = tqdm(val_loader, file=sys.stdout)

    preds = []
    labels = []

    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            with torch.cuda.amp.autocast():
                pred = model(*inputs)

            preds.append(pred.detach().cpu().numpy().ravel())
            labels.append(target.detach().cpu().numpy().ravel())

    return np.concatenate(labels), np.concatenate(preds)

In [13]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
print(device)

True
1
Graphics Device
cuda


In [16]:
# import finetuned model

model_path = f'./model/pretrain/codebert-base' # origin model path
ckpt_path = os.path.join(f'./model/finetuned/20220703/outputs', 'pytorch_model_3.bin') # 체크포인트 path

print(model_path, '\n', ckpt_path)
model = MarkdownModel(model_path)

# 불러온 모델의 weight를 finetuned된 모델로 update해줌
model.load_state_dict(torch.load(ckpt_path))
model.eval()

./model/pretrain/codebert-base 
 ./model/finetuned/20220703/outputs/pytorch_model_3.bin


MarkdownModel(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,

In [17]:
print(device)
model = model.to(device)

cuda


In [18]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [39]:
# Valid Set 결과 추출
# codeBERT & Valid(167,313건) 기준 약 16분 걸림
y_label, y_pred = validate(model, val_loader)

100%|██████████| 20915/20915 [16:39<00:00, 20.92it/s]


In [40]:
val_df_mark.shape, y_label.shape, y_pred.shape

((167313, 8), (167313,), (167313,))

In [41]:
# Valid 셋에 label, pred 추가
val_df_mark.loc[val_df_mark["cell_type"] == "markdown", "label"] = y_label
val_df_mark.loc[val_df_mark["cell_type"] == "markdown", "pred"] = y_pred

- encode 되어있는 ds 변환
    - 해당 작업이 조금 오래걸림

In [129]:
# incode Data append
tokenizer_forex = AutoTokenizer.from_pretrained(tokenizer_path)

decode_lst = []

for i in tqdm(range(len(val_ds))):
    row = val_ds.__getitem__(i)
    _encode = row[0]
    _decode = tokenizer_forex.decode(_encode).replace('<pad>', " ")
#     print(_encode)
#     print(_decode)
#     print(row)
    decode_lst.append(_decode)
len(decode_lst)

100%|██████████| 167313/167313 [21:05<00:00, 132.21it/s]


167313

In [134]:
val_df_mark.loc[:,'input_text'] = decode_lst
val_df_mark.head(2)

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred,input_text
0,000597ac4c6700,a3e03a94,markdown,"Always remember to delete your -now- useless data, will free up some memory.",30,81ab937c,NaN,0.666667,0.666667,0.741699,"<s>Always remember to delete your -now- useless data, will free up some memory.</s> ..."
1,000597ac4c6700,8e68c8ad,markdown,And it's all encoded. We only encoded these columns to get rid of the string data type but **if you want to train a ...,23,81ab937c,NaN,0.511111,0.511111,0.563965,<s>And it's all encoded. We only encoded these columns to get rid of the string data type but **if you want to train...


In [136]:
# Valid 셋에 label, pred 추가
use_col = ['cell_id', 'label', 'pred', 'input_text']

new_val_df = val_df.merge(val_df_mark[use_col]
                         ,how = 'left'
                         ,on = 'cell_id').copy()

print(val_df.shape, new_val_df.shape)
display(new_val_df.head(2))
display(new_val_df.tail(2))

(490240, 8) (490242, 11)


,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred,input_text
0,000597ac4c6700,ede4241f,code,import pandas as pd\nimport numpy as np,1,81ab937c,NaN,0.022222,NaN,NaN,NaN
1,000597ac4c6700,36b989a3,code,"## Function to reduce the DF size\ndef reduce_mem_usage(df, verbose=True):\n numerics = ['int16', 'int32', 'int64...",3,81ab937c,NaN,0.066667,NaN,NaN,NaN


,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred,input_text
490240,fff6c12f17ac92,7912edbd,code,"sample_df.to_csv('submission.csv',index=False)",25,41d132dc,NaN,0.925926,NaN,NaN,NaN
490241,fff6c12f17ac92,19a20bdc,markdown,"**If you like it , please upvote :)**",26,41d132dc,NaN,0.962963,0.962963,1.036133,"<s>**If you like it, please upvote :)**</s> <s># Ignore the warni..."


In [138]:
# code cell label 추가
new_val_df.label = np.where(new_val_df.label.isna(), new_val_df.pct_rank, new_val_df.label)
new_val_df.pred = np.where(new_val_df.pred.isna(), new_val_df.pct_rank, new_val_df.pred)

new_val_df

,id,cell_id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred,input_text
0,000597ac4c6700,ede4241f,code,import pandas as pd\nimport numpy as np,1,81ab937c,NaN,0.022222,0.022222,0.022222,NaN
1,000597ac4c6700,36b989a3,code,"## Function to reduce the DF size\ndef reduce_mem_usage(df, verbose=True):\n numerics = ['int16', 'int32', 'int64...",3,81ab937c,NaN,0.066667,0.066667,0.066667,NaN
2,000597ac4c6700,b91fc1a4,code,"train = pd.read_csv(""/kaggle/input/bdg2-class-competition/train.csv"")\ntest = pd.read_csv(""/kaggle/input/bdg2-class-...",5,81ab937c,NaN,0.111111,0.111111,0.111111,NaN
3,000597ac4c6700,97c1bb21,code,train = reduce_mem_usage(train)\ntest = reduce_mem_usage(test)\nwtrain = reduce_mem_usage(wtrain)\nwtest = reduce_me...,7,81ab937c,NaN,0.155556,0.155556,0.155556,NaN
4,000597ac4c6700,148274fd,code,metadata.info(),8,81ab937c,NaN,0.177778,0.177778,0.177778,NaN
...,...,...,...,...,...,...,...,...,...,...,...
490237,fff6c12f17ac92,bccd1fcb,code,"learn.data.add_test(ImageList.from_df(sample_df,'../input/aptos2019-blindness-detection',folder='test_images',suffix...",22,41d132dc,NaN,0.814815,0.814815,0.814815,NaN
490238,fff6c12f17ac92,323f0b85,code,"preds,y = learn.get_preds(DatasetType.Test)",23,41d132dc,NaN,0.851852,0.851852,0.851852,NaN
490239,fff6c12f17ac92,ac1e1d7f,code,sample_df.diagnosis = preds.argmax(1)\nsample_df.head(),24,41d132dc,NaN,0.888889,0.888889,0.888889,NaN
490240,fff6c12f17ac92,7912edbd,code,"sample_df.to_csv('submission.csv',index=False)",25,41d132dc,NaN,0.925926,0.925926,0.925926,NaN


## 결과물 print

In [139]:
#metric.py
from bisect import bisect

# row 단위로 kendall tau print
def kendall_tau_byrow(gt, pred):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    
    ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
    total_inversions += count_inversions(ranks)
    n = len(gt)
    total_2max += n * (n - 1)
    return abs(1 - 4 * total_inversions / total_2max)


In [48]:
# Valid set 성능 확인
tmp_val_df = val_df.copy()
tmp_val_df["pred"] = tmp_val_df.groupby(["id", "cell_type"])["rank"].rank(pct=True) #그룹별 순위 구하기, 큰 순서대로 정렬
tmp_val_df.loc[tmp_val_df["cell_type"] == "markdown", "pred"] = y_pred              #마크다운엔 prediction 값
y_dummy = tmp_val_df.sort_values("pred").groupby('id')['cell_id'].apply(list)
print("Valid score", kendall_tau(df_orders.loc[y_dummy.index], y_dummy)) # label, pred 순으로 insert
del tmp_val_df

Valid score 0.8489752778268465


In [58]:
# id별로 성능이 낮은건 확인

label_byid = df_orders.loc[y_dummy.index].reset_index(drop=False)
pred_byid = y_dummy.reset_index(drop=False)
label_byid.columns = ['id', 'label_order']
pred_byid.columns = ['id', 'pred_order']

display(label_byid.shape, label_byid.head(2)
        , pred_byid.shape, pred_byid.head(2))

total_result = label_byid.merge(pred_byid
                               ,how = 'left'
                               ,on = 'id')
print('\n\n',total_result.shape)
total_result.head(2)

(10520, 2)

,id,label_order
0,000597ac4c6700,"[ba54a747, ede4241f, fb42ece2, 36b989a3, 2fa559cb, b91fc1a4, 649083d4, 97c1bb21, 148274fd, 7f40f579, 92a7ccd5, 9ed48..."
1,000a4651cce8f4,"[2844b16f, 883db462, 9ae35818, f1dee85b, 00e71412, 3ec4ab79, 7bfe7649, 5bcfa9ad, be38feb5, f417baab, cf43b3ce, edbdd..."


(10520, 2)

,id,pred_order
0,000597ac4c6700,"[ba54a747, ede4241f, fb42ece2, 36b989a3, 2fa559cb, b91fc1a4, 649083d4, 97c1bb21, 148274fd, 7f40f579, 92a7ccd5, a791b..."
1,000a4651cce8f4,"[2844b16f, 883db462, 9ae35818, f1dee85b, 3ec4ab79, 7bfe7649, be38feb5, 00e71412, 5bcfa9ad, cf43b3ce, f417baab, edbdd..."




 (10520, 3)


,id,label_order,pred_order
0,000597ac4c6700,"[ba54a747, ede4241f, fb42ece2, 36b989a3, 2fa559cb, b91fc1a4, 649083d4, 97c1bb21, 148274fd, 7f40f579, 92a7ccd5, 9ed48...","[ba54a747, ede4241f, fb42ece2, 36b989a3, 2fa559cb, b91fc1a4, 649083d4, 97c1bb21, 148274fd, 7f40f579, 92a7ccd5, a791b..."
1,000a4651cce8f4,"[2844b16f, 883db462, 9ae35818, f1dee85b, 00e71412, 3ec4ab79, 7bfe7649, 5bcfa9ad, be38feb5, f417baab, cf43b3ce, edbdd...","[2844b16f, 883db462, 9ae35818, f1dee85b, 3ec4ab79, 7bfe7649, be38feb5, 00e71412, 5bcfa9ad, cf43b3ce, f417baab, edbdd..."


In [60]:
# 결과물에 kendall_score 추가
total_result.loc[:,'kendall_score'] = [kendall_tau_byrow(n[0], n[1]) for n in total_result[['label_order', 'pred_order']].values]
total_result.head(2)

,id,label_order,pred_order,kendall_score
0,000597ac4c6700,"[ba54a747, ede4241f, fb42ece2, 36b989a3, 2fa559cb, b91fc1a4, 649083d4, 97c1bb21, 148274fd, 7f40f579, 92a7ccd5, 9ed48...","[ba54a747, ede4241f, fb42ece2, 36b989a3, 2fa559cb, b91fc1a4, 649083d4, 97c1bb21, 148274fd, 7f40f579, 92a7ccd5, a791b...",0.957576
1,000a4651cce8f4,"[2844b16f, 883db462, 9ae35818, f1dee85b, 00e71412, 3ec4ab79, 7bfe7649, 5bcfa9ad, be38feb5, f417baab, cf43b3ce, edbdd...","[2844b16f, 883db462, 9ae35818, f1dee85b, 3ec4ab79, 7bfe7649, be38feb5, 00e71412, 5bcfa9ad, cf43b3ce, f417baab, edbdd...",0.908832


In [61]:
# kendall tau score 분포 확인
total_result.kendall_score.describe()

count    10520.000000
mean         0.881397
std          0.105744
min          0.000000
25%          0.838235
50%          0.907179
75%          0.955556
max          1.000000
Name: kendall_score, dtype: float64

In [63]:
tmp_tb = new_val_df[new_val_df.id == '05f96edae0f024'].copy()
tmp_tb = tmp_tb.set_index('cell_id')

# tmp_tb.loc[['9d98338d', 'ee86eac5']]

In [ ]:
import seaborn as sns
sns.distplot(total_result.kendall_score)

### kendall tau score가 낮은 id sampleing

In [ ]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

print(color.YELLOW + 'Hello World !' + color.END)

In [104]:
low_score_sample = total_result.sort_values('kendall_score').iloc[:10,:]
display(low_score_sample)

# 실제 텍스트 확인
def print_result(cell_tb, notebook_id, label_order, pred_order):
    # cell tb 중 확인할 notebook_id sort
    tmp_tb = cell_tb[cell_tb.id == notebook_id].copy()
    tmp_tb = tmp_tb.set_index('cell_id')

    print(f'>>> ---------- Notebook id: {notebook_id} ----------')

#     # label 순서 대로 cell print
#     print(f'>>> label_order result')
#     display(tmp_tb.loc[label_order])
#     for t in tmp_tb.loc[label_order].source.tolist():
#         display(t)
    
    # pred order 대로 cell print
    print(f'>>> pred_order_col result')
    display(tmp_tb.loc[pred_order])
#     for t in tmp_tb.loc[pred_order].source.tolist():
#         display(t)
    
for row in low_score_sample.iterrows():
    row = row[1]
    print(row)
    n_id = row.id
    label_order = row.label_order
    pred_order = row.pred_order
    print_result(new_val_df, n_id, label_order, pred_order)
    print('\n\n\n')


,id,label_order,pred_order,kendall_score
2643,3fd66fe0a96d18,"[8538920c, c4d2907b, c142a2ee, 136f872b]","[c4d2907b, 136f872b, 8538920c, c142a2ee]",0.000000
3526,544655921beef7,"[048e2d9b, bef56024, ea556589, 20d3fe90, 69294a5a, 4b60c22a, 01f6f76b, 06d7f1db, 42485723, 9b8b36c4, 0cf7ee49, 625f2...","[20d3fe90, db4baf15, ea556589, 3cf1066f, fdb913ac, c90e3bd6, bc18d466, ba162cd7, 76a0e541, 23597b53, bf1dc93b, 4b60c...",0.025309
1996,2fa3c4c997e1d1,"[b299f9ff, 0f8c59a6, 0cd37fbf, 41cac302, 2770c4e2, f8c8c680, 39d7665b, 3919bd9a, 485ecbae, 2c527861, 8c1c0e0a, fc058...","[0cd37fbf, e0769a65, da72def6, 79b32d5e, 9432920a, 44280a92, f4a41dbe, e752cb02, cb699fd2, b299f9ff, 0f8c59a6, f54b8...",0.086542
8038,c27850f9bbcac4,"[92e0a22c, 6574991c, aa24cfb2, 1efd2c07, f87c1725, e7529972, db582ff6, f34383a6, d1866a2a, 2cec003c, 72b3c352, 5a932...","[92e0a22c, 1c7d3327, 5a932c2f, f87c1725, 6574991c, aa24cfb2, 49831d80, db582ff6, 2cec003c, d1866a2a, d4d1bfee, 72b3c...",0.116667
3963,5e97933310f217,"[46fa93e3, 8515c21d, b0ea29f4, d72d0507, 553f5b96, eb2b787d, c926405c, dbbfaf24, 2708c4b8, 96b20353, 6934a531, 73da1...","[1b79d054, 2d7d9227, 9f5e2233, 46fa93e3, b0ea29f4, c472f10d, 8515c21d, d72d0507, 9c617cba, dbbfaf24, f907fbe0, 2708c...",0.131183
3635,56a364c9726435,"[d1824dcf, 232912f8, ddfe5a63, 71945f79, 7cd943a6, 2b918080, c2978663]","[2b918080, 232912f8, ddfe5a63, 71945f79, 7cd943a6, d1824dcf, c2978663]",0.142857
8650,d127172adeacea,"[eebc3086, 722cf377, b37481da, 712782ed, 1afd3599, 0315d35b, e92fec1b, ceb68917, b53f604a, 01514c28, 5ad53400, c2a6e...","[eebc3086, 712782ed, 5ad53400, 722cf377, b53f604a, e92fec1b, c2a6e274, 01514c28, 1afd3599, ceb68917, 0315d35b, 56f0a...",0.179487
3093,4a99d2ffd04cd6,"[19bb0372, 782c7b9e, 5b019082, 067348b8, 531994ce]","[531994ce, 19bb0372, 782c7b9e, 5b019082, 067348b8]",0.200000
1353,206aadfedae1ad,"[46946ca5, 022d6bd1, 3a917eba, 59da069c, 4d9f8b9f, eb012803, d4470b3c, 6296022f, 2ba8a5bf, 86943f9e, 9927ec8f, 37578...","[46946ca5, 59da069c, 77022ccd, a2422444, 7d3549e7, 55790065, 1e7ecfb4, eb012803, 3ec5ab6f, 3757841d, 1e6707b1, 022d6...",0.219873
984,17845073a429f3,"[502033a4, 794feada, a42f4b7e, b9d3bfc2, a706930b, 1441e3fc, ece1fb3b, d4e523f0, c6b9d271, 87794ae1, bc9d7947, 1e2a3...","[502033a4, d50e67bb, a42f4b7e, 205b4129, 7cc1fd2a, 68bcf209, 794feada, b9d3bfc2, a706930b, 1441e3fc, a6f2a408, d4e52...",0.224638


id                                         3fd66fe0a96d18
label_order      [8538920c, c4d2907b, c142a2ee, 136f872b]
pred_order       [c4d2907b, 136f872b, 8538920c, c142a2ee]
kendall_score                                         0.0
Name: 2643, dtype: object
>>> ---------- Notebook id: 3fd66fe0a96d18 ----------
>>> pred_order_col result


,id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred
cell_id,,,,,,,,,
c4d2907b,3fd66fe0a96d18,markdown,# Title,1,0e15379d,NaN,0.25,0.25,0.174438
136f872b,3fd66fe0a96d18,markdown,title,3,0e15379d,NaN,0.75,0.75,0.345459
8538920c,3fd66fe0a96d18,code,"print(""hello world"")",0,0e15379d,NaN,0.00,0.00,0.000000
c142a2ee,3fd66fe0a96d18,code,title,2,0e15379d,NaN,0.50,0.50,0.500000






id                                                                                                                        544655921beef7
label_order      [048e2d9b, bef56024, ea556589, 20d3fe90, 69294a5a, 4b60c22a, 01f6f76b, 06d7f1db, 42485723, 9b8b36c4, 0cf7ee49, 625f2...
pred_order       [20d3fe90, db4baf15, ea556589, 3cf1066f, fdb913ac, c90e3bd6, bc18d466, ba162cd7, 76a0e541, 23597b53, bf1dc93b, 4b60c...
kendall_score                                                                                                                   0.025309
Name: 3526, dtype: object
>>> ---------- Notebook id: 544655921beef7 ----------
>>> pred_order_col result


,id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred
cell_id,,,,,,,,,
20d3fe90,544655921beef7,markdown,# Explain inner working on logistic regression\nhttps://satishgunjal.com/binary_lr/,3,035e1579,NaN,0.037037,0.037037,-0.003082
db4baf15,544655921beef7,markdown,"# Difference between univariate, bivariate and multivariate analysis?\n\n* Univariate Analysis\n\n![Univariate_Analy...",41,035e1579,NaN,0.506173,0.506173,-0.002829
ea556589,544655921beef7,markdown,# Explain inner working on linear regression\nhttps://satishgunjal.com/univariate_lr/,2,035e1579,NaN,0.024691,0.024691,-0.002739
3cf1066f,544655921beef7,markdown,"# Explain dimensionality reduction, and its benefits?\n* Dimensionality reduction referes to the process of converti...",54,035e1579,NaN,0.666667,0.666667,-0.001030
fdb913ac,544655921beef7,markdown,"# Explain Principal Componenet Analysis?\n* Principal Component Analysis (PCA) is dimensionality reduction method, t...",64,035e1579,NaN,0.790123,0.790123,-0.000325
...,...,...,...,...,...,...,...,...,...
0c99587e,544655921beef7,markdown,# Explain the scenario where both false positive and false negative are equally important\n* In banking industry giv...,77,035e1579,NaN,0.950617,0.950617,0.750000
048e2d9b,544655921beef7,markdown,"# Important Tips\n* Datascience interview questions can include questions from statistics, math, data visualization,...",0,035e1579,NaN,0.000000,0.000000,0.831055
9b8b36c4,544655921beef7,markdown,# What is more important model accuracy or model performance?\n* Short answer is: Model accuracy matters the most! i...,9,035e1579,NaN,0.111111,0.111111,0.833496


...

,id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred
cell_id,,,,,,,,,
531994ce,4a99d2ffd04cd6,markdown,Credits: Python Engineer\n\nDon't hesitate to watch YouTube video. It has clear explanation\n\nhttps://youtu.be/3Kb0...,4,9ef2154d,NaN,0.8,0.8,-0.001771
19bb0372,4a99d2ffd04cd6,code,"import torch\n\n\n# Intialize values to X,Y\n\n\nx = torch.tensor(1.0)\ny = torch.tensor(2.0)",0,9ef2154d,NaN,0.0,0.0,0.000000
782c7b9e,4a99d2ffd04cd6,code,"\n# This is the parameter we want to optimize -> requires_grad=True\nw = torch.tensor(1.0, requires_grad=True)\n\n# ...",1,9ef2154d,NaN,0.2,0.2,0.200000
5b019082,4a99d2ffd04cd6,code,\n\n# backward pass to compute gradient dLoss/dw\nloss.backward()\nprint(w.grad),2,9ef2154d,NaN,0.4,0.4,0.400000
067348b8,4a99d2ffd04cd6,code,"\n\n# update weights\n# next forward and backward pass...\n\n# continue optimizing:\n# update weights, this operatio...",3,9ef2154d,NaN,0.6,0.6,0.600000






id                                                                                                                        206aadfedae1ad
label_order      [46946ca5, 022d6bd1, 3a917eba, 59da069c, 4d9f8b9f, eb012803, d4470b3c, 6296022f, 2ba8a5bf, 86943f9e, 9927ec8f, 37578...
pred_order       [46946ca5, 59da069c, 77022ccd, a2422444, 7d3549e7, 55790065, 1e7ecfb4, eb012803, 3ec5ab6f, 3757841d, 1e6707b1, 022d6...
kendall_score                                                                                                                   0.219873
Name: 1353, dtype: object
>>> ---------- Notebook id: 206aadfedae1ad ----------
>>> pred_order_col result


,id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred
cell_id,,,,,,,,,
46946ca5,206aadfedae1ad,markdown,# 7 Coolest Python Packages Top Kagglers Are Using Without Telling You\n## Let me expose the secrets...\n![](https:/...,0,54b0f8b3,NaN,0.000000,0.000000,-0.001430
59da069c,206aadfedae1ad,markdown,"Kaggle is a hot spot for what is trending in data science and machine learning.\n\nDue to its competitiveness, the t...",3,54b0f8b3,NaN,0.068182,0.068182,0.005646
77022ccd,206aadfedae1ad,markdown,The available tools and packages to execute data science tasks are endless. Everyone has the right to be overwhelmed...,42,54b0f8b3,NaN,0.954545,0.954545,0.017120
a2422444,206aadfedae1ad,markdown,Explainable AI (XAI) is one of the strongest trends in the ML and AI sphere. Companies and businesses are starting t...,32,54b0f8b3,NaN,0.727273,0.727273,0.018707
7d3549e7,206aadfedae1ad,markdown,"Normally, I am against any library or tool that takes a programmer away from writing actual code. But, since auto-ED...",40,54b0f8b3,NaN,0.909091,0.909091,0.028137
55790065,206aadfedae1ad,markdown,"Lazypredict is one of the best one-liner packages I have ever seen.\n\nUsing the library, you can train almost all S...",20,54b0f8b3,NaN,0.454545,0.454545,0.028152
1e7ecfb4,206aadfedae1ad,markdown,### 🛠 GitHub and documentation\n- https://github.com/h2oai/datatable\n- https://datatable.readthedocs.io/en/latest/?...,13,54b0f8b3,NaN,0.295455,0.295455,0.028641
eb012803,206aadfedae1ad,markdown,"![](https://cdn-images-1.medium.com/max/800/1*1sUCX4FKCLjJzbEqMjg2YQ.png)\n<figcaption style=""text-align: center;"">\...",5,54b0f8b3,NaN,0.113636,0.113636,0.041168
3ec5ab6f,206aadfedae1ad,markdown,![](https://cdn-images-1.medium.com/max/800/1*ZoH5jGuQiKKhxeZYNHcwDA.png),36,54b0f8b3,NaN,0.818182,0.818182,0.064270






id                                                                                                                        17845073a429f3
label_order      [502033a4, 794feada, a42f4b7e, b9d3bfc2, a706930b, 1441e3fc, ece1fb3b, d4e523f0, c6b9d271, 87794ae1, bc9d7947, 1e2a3...
pred_order       [502033a4, d50e67bb, a42f4b7e, 205b4129, 7cc1fd2a, 68bcf209, 794feada, b9d3bfc2, a706930b, 1441e3fc, a6f2a408, d4e52...
kendall_score                                                                                                                   0.224638
Name: 984, dtype: object
>>> ---------- Notebook id: 17845073a429f3 ----------
>>> pred_order_col result


,id,cell_type,source,rank,ancestor_id,parent_id,pct_rank,label,pred
cell_id,,,,,,,,,
502033a4,17845073a429f3,markdown,<h1> COVID-19 Analysis Notebook on Comorbidities (CANCe) </h1>\n\n<h2> What is the incident of COVID-19 Infections i...,0,fbae70fc,NaN,0.000000,0.000000,-0.003014
d50e67bb,17845073a429f3,markdown,"<img src=""https://www.statista.com/graphic/1/1102796/south-korea-covid-19-deaths-by-chronic-disease.jpg"" alt=""Statis...",18,fbae70fc,NaN,0.750000,0.750000,0.057007
a42f4b7e,17845073a429f3,markdown,# <a id='main'><h3>Table of Contents</h3></a>\n- [Importing the Essential Libraries](#lib)\n- [Datasets used in note...,2,fbae70fc,NaN,0.083333,0.083333,0.057831
205b4129,17845073a429f3,markdown,<h3> Breakdown of COVID-19 Death Cases in South Korea </h3>\n\nThe following is the dataset that shows the deaths in...,17,fbae70fc,NaN,0.708333,0.708333,0.095581
7cc1fd2a,17845073a429f3,markdown,"<img src=""https://www.statista.com/graphic/1/1110949/common-comorbidities-in-covid-19-deceased-patients-in-italy.jpg...",22,fbae70fc,NaN,0.916667,0.916667,0.121704
68bcf209,17845073a429f3,markdown,"<img src=""https://www.statista.com/graphic/1/1108836/china-coronavirus-covid-19-fatality-rate-by-health-condition.jp...",20,fbae70fc,NaN,0.833333,0.833333,0.134521
794feada,17845073a429f3,markdown,<H3> Task Details </H3>\n\nThe Roche Data Science Coalition is a group of like-minded public and private organizatio...,1,fbae70fc,NaN,0.041667,0.041667,0.145874
b9d3bfc2,17845073a429f3,markdown,# <a id='lib'><h3>Importing the essential libraries</h3></a>,3,fbae70fc,NaN,0.125000,0.125000,0.155273
a706930b,17845073a429f3,code,#Data Analyses Libraries\nimport pandas as pd \nimport numpy as np \nfrom urllib.request import ur...,4,fbae70fc,NaN,0.166667,0.166667,0.166667


### kendall tau score가 낮은 id 실제 print되는 STT 확인

In [99]:
low_score_sample = total_result.sort_values('kendall_score').iloc[:10,:]
# display(low_score_sample)

# 실제 인코딩 텍스트 확인
def print_incoding_result(cell_tb, notebook_id, label_order, pred_order):
    # cell tb 중 확인할 notebook_id sort
    tmp_tb = cell_tb[cell_tb.id == notebook_id].copy()
    tmp_tb = tmp_tb.set_index('cell_id')

    print(f'################### Notebook id: {notebook_id} ###################\n')

    # pred order 대로 cell print
#     print(f'>>> pred_order_col result')
#     display(tmp_tb.loc[pred_order])
    for c_type, label_r, pred_r, t, e_t in tmp_tb.loc[pred_order][['cell_type','rank', 'pred', 'source', 'input_text']].values:
        if c_type == 'markdown':
            print(f'------------------------- code start--------------------------------------------------------------------------------------')
            print(f'{color.YELLOW}1. Code Type:{color.END}{c_type}\n')
            print(f'{color.YELLOW}>> Label rank:{color.END}{label_r}\n')
            print(f'{color.YELLOW}>> Pred rank:{color.END}{pred_r}\n')     
            print(f'{color.YELLOW}2. og text:{color.END} \n{t}\n')
            print(f'{color.YELLOW}3. input_text:{color.END} \n{e_t}\n')
#             _encode = tokenizer_forex.encode(t, max_length=md_max_len)
#             print(f'4. encode: \n{_encode[:5]}\n')        
#             print(f'5. decode: \n{tokenizer_forex.decode(_encode)}')
            print(f'------------------------- code end--------------------------------------------------------------------------------------')        
        else:
            pass
    return None
print(md_max_len)

64
################### Notebook id: 2fa3c4c997e1d1 ###################

------------------------- code start--------------------------------------------------------------------------------------
1. Code Type: markdown

2. og_source: 
Keras 전처리 층 API는 개발자들이 Keras 고유의 입력 처리 파이프라인을 만들 수 있게 해줍니다. 이 입력 처리 파이프라인들은 Keras가 아닌 작업 흐름 안에서 독립적인 사전 처리 코드로써 사용되고, 직접적으로 Keras 모델들과 결합되고, Keras SavedModel의 일부로써 수출될 수 있습니다.



'2. og_source: \nKeras 전처리 층 API는 개발자들이 Keras 고유의 입력 처리 파이프라인을 만들 수 있게 해줍니다. 이 입력 처리 파이프라인들은 Keras가 아닌 작업 흐름 안에서 독립적인 사전 처리 코드로써 사용되고, 직접적으로 Keras 모델들과 결합되고, Keras SavedModel의 일부로써 수출될 수 있습니다.\n'

3. encode: 
[0, 530, 254, 281, 46747]

4. decode: 
<s>Keras 전처리 층 API는 개발자들이 Keras 고유의 입력 처리 파�</s>
------------------------- code start--------------------------------------------------------------------------------------
------------------------- code start--------------------------------------------------------------------------------------
1. Code Type: markdown

2. og_source: 
[바닥에서 부터 이미지 분류](https://keras.io/examples/vision/image_classification_from_scratch/) 예제에서 비슷한 설정 활동을 볼 수 있습니다.



'2. og_source: \n[바닥에서 부터 이미지 분류](https://keras.io/examples/vision/image_classification_from_scratch/) 예제에서 비슷한 설정 활동을 볼 수 있습니다.\n'

3. encode: 
[0, 10975, 45209, 7487, 10674]

4. decode: 
<s>[바닥에서 부터 이미지 분류](https://keras.io/examples/vision/image_classification_from_scratch/) 예제</s>
------------------------- code start--------------------------------------------------------------------------------------
------------------------- code start--------------------------------------------------------------------------------------
1. Code Type: markdown

2. og_source: 
[바닥에서 부터 텍스트 분류](https://keras.io/examples/nlp/text_classification_from_scratch/) 예제에서 `Embedding` 방식과 결합된 `TextVectorization` 층을 동작 속에서 볼 수 있습니다.



...

3. encode: 
[0, 10431, 46747, 18537, 8906]

4. decode: 
<s># 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다
data =</s>
------------------------- code start--------------------------------------------------------------------------------------
------------------------- code start--------------------------------------------------------------------------------------
1. Code Type: code

2. og_source: 
# 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# "binary" 출력 방식(multi-hot)과 ngram=2(모든 bigram 색인)으로
# TextVectorization을 생성합니다
text_vectorizer = preprocessing.TextVectorization(output_mode="binary", ngrams=2)
# `adapt()`를 통해 bigram들을 색인합니다
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Dense 모델을 생성합니다
inputs = keras.Input(shape=(1,), dtyp

'2. og_source: \n# 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다\ndata = tf.constant(\n    [\n        "The Brain is wider than the Sky",\n        "For put them side by side",\n        "The one the other will contain",\n        "With ease and You beside",\n    ]\n)\n# "binary" 출력 방식(multi-hot)과 ngram=2(모든 bigram 색인)으로\n# TextVectorization을 생성합니다\ntext_vectorizer = preprocessing.TextVectorization(output_mode="binary", ngrams=2)\n# `adapt()`를 통해 bigram들을 색인합니다\ntext_vectorizer.adapt(data)\n\nprint(\n    "Encoded text:\\n",\n    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),\n    "\\n",\n)\n\n# Dense 모델을 생성합니다\ninputs = keras.Input(shape=(1,), dtype="string")\nx = text_vectorizer(inputs)\noutputs = layers.Dense(1)(x)\nmodel = keras.Model(inputs, outputs)\n\n# (알 수 없는 토큰을 포함하는) 평가 데이터 위에서 모델을 호출합니다\ntest_data = tf.constant(["The Brain is deeper than the sea"])\ntest_output = model(test_data)\n\nprint("Model output:", test_output)\n'

3. encode: 
[0, 10431, 46747, 18537, 8906]

4. decode: 
<s># 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다
data =</s>
------------------------- code start--------------------------------------------------------------------------------------
------------------------- code start--------------------------------------------------------------------------------------
1. Code Type: code

2. og_source: 
# 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# "tf-idf" 출력 방식(TF-IDF 가중치를 사용하는 multi-hot)과 ngram=2(모든 bigram 색인)으로
# TextVectorization을 생성합니다
text_vectorizer = preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)
# `adapt()`를 통해 bigram들을 색인합니다
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Dense 모델을 생성합니다
inputs = keras.Input

'2. og_source: \n# 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다\ndata = tf.constant(\n    [\n        "The Brain is wider than the Sky",\n        "For put them side by side",\n        "The one the other will contain",\n        "With ease and You beside",\n    ]\n)\n# "tf-idf" 출력 방식(TF-IDF 가중치를 사용하는 multi-hot)과 ngram=2(모든 bigram 색인)으로\n# TextVectorization을 생성합니다\ntext_vectorizer = preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)\n# `adapt()`를 통해 bigram들을 색인합니다\ntext_vectorizer.adapt(data)\n\nprint(\n    "Encoded text:\\n",\n    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),\n    "\\n",\n)\n\n# Dense 모델을 생성합니다\ninputs = keras.Input(shape=(1,), dtype="string")\nx = text_vectorizer(inputs)\noutputs = layers.Dense(1)(x)\nmodel = keras.Model(inputs, outputs)\n\n# (알 수 없는 토큰을 포함하는) 평가 데이터 위에서 모델을 호출합니다\ntest_data = tf.constant(["The Brain is deeper than the sea"])\ntest_output = model(test_data)\n\nprint("Model output:", test_output)\n'

3. encode: 
[0, 10431, 46747, 18537, 8906]

4. decode: 
<s># 층에 adapt하기 위한 몇가지 텍스트 데이터를 정의합니다
data =</s>
------------------------- code start--------------------------------------------------------------------------------------






In [ ]:
idx = 3
row = low_score_sample.iloc[idx]
n_id = row.id
label_order = row.label_order
pred_order = row.pred_order
print_incoding_result(new_val_df, n_id, label_order, pred_order)
print('\n\n\n')